In [ ]:
import os
import shutil
import re
import json

def list_files_by_folder(directory):
    files_by_folder = []
    for root, dirs, files in os.walk(directory):
        folder_files = []
        for file in files:
            folder_files.append(os.path.join(root, file))
        files_by_folder.append(folder_files)
    return files_by_folder


directory = "/workspace/KoLLaVA/data/train"
files_by_folder = list_files_by_folder(directory)

i = 0
j = 0
result = '''[
'''
for idx, source_files in enumerate(files_by_folder):
    files = sorted(source_files)

    for file_name in files:
        ##### "image" : {images}
        ##### "action_uid" : {action_uid}
        ##### "from" : "goal" , "value" : {<image>\n goal}
        file_path = '/'.join(file_name.split('/')[:-1])       
        name_list = file_name.split('/')[-1].split('_')
        name_li = name_list[2].split('.')[0]

        images = file_name.split('/')[-1].split('.')[0]
        action_uid = name_list[0]
        goal = f'{name_list[1]}'#_{name_li}'
        # print(f'{images}.jpg')
        # print(action_uid)
        # print(goal)



        ##### "from" : "answer_kr", "value" : ["action_type": "", "touch_point": "[y, x]", "lift_point": "[y, x]", "typed_text": "", "screen_description": "", "action_description":""]
        # print(file_name)
        inside = open(file_name, 'r', encoding='utf-8')
        inside = inside.readline().rstrip()
        inside_list = inside.split(' ')
        # print(inside_list)
        
        action_type = f"{inside_list[0]}"
        typed_text = inside_list[1]
        if inside_list[2] == '-1' and inside_list[3] == '-1' :
            Tpoint_y = -1.0
            Tpoint_x = -1.0
        else :
            Tpoint_y = round(float(inside_list[2])*1080, 1)
            Tpoint_x = round(float(inside_list[3])*1920, 1)
            
        touch_point = f"[{Tpoint_y}, {Tpoint_x}]"
        # print(touch_point)

        screen_description = inside_list[6]
        action_description = inside_list[7]

        value_ex = f'[\\"action_type\\": \\"{action_type}\\", \\"touch_point\\": \\"{touch_point}\\", \\"lift_point\\": \\"{touch_point}\\", \\"typed_text\\": \\"{typed_text}\\", \\"screen_description\\": \\"{screen_description}\\", \\"action_description\\": \\"{action_description}\\"]'
        # print(value_ex)

        result_1 = '''    { '''
        result_2 = f'''        "image" : "{images}.jpg",'''
        result_3 = f'''        "action_uid" :"{action_uid}",'''
        result_4 = f'''        "conversations" : ['''
        result_5 = '''            {
                "from" :  "goal",'''
        result_6 = '''                "value" : "<image>\\n'''+f'{goal}"'
        result_7 = '''            },
            {
                "from" : "answer_kr", '''
        result_8 = f'''                "value" : "{value_ex}"'''
        result_9 = '''
            }
        ]
    },
'''
        result_all = result_1+'\n'+result_2+'\n'+result_3+'\n'+result_4+'\n'+result_5+'\n'+result_6+'\n'+result_7+'\n'+result_8+'\n'+result_9
        result += result_all
    #     i += 1
    #     if i == 20 :
    #         break
    # j += 1
    # if i == 20 :
    #     break

# print(result)
with open('/workspace/test_data.txt', 'w', encoding='utf_8') as file :
     file.write(result)
print('done')